In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time

# ----------------------------------
# 날짜 입력받기
#   -> 입력하지 않으면 오늘날짜 반환
#   -> 잘못된 형식이면 None 반환
# ----------------------------------
def date_input():

    date_str = input("날짜:(형식:2020-01-05)")
    
    if date_str=="":
        date_str = datetime.now().strftime('%Y-%m-%d')
    
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
    except:
        print("날짜 형색이 올바르지 않습니다.")
        date_str = None
    
    return date_str

date = date_input()


date


'2025-04-23'

In [30]:

# ----------------------------------
# 웹페이지 요청하여 응답객체 반환
# ----------------------------------
url = 'https://finance.naver.com/news/mainnews.naver?date='+date
response = requests.get(url)
    
# ----------------------------------
# 웹페이지 파싱하여 BeautifulSoup 객체 반환
# ----------------------------------
soup = BeautifulSoup(response.text, 'html.parser')

# ----------------------------------
# article 리스트 추출
# ----------------------------------   
articles = soup.select(".block1")

# ----------------------------------
# article 리스트에서 요소의 텍스트, 속성 추출
# ----------------------------------   
data = []
for article in articles:
    title = article.select_one(".articleSubject>a").text
    summary = article.select_one(".articleSummary").contents[0].text.strip()
    press = article.select_one(".press").text.strip()
    wdate = article.select_one(".wdate").text.strip()
    link = article.select_one(".articleSubject>a").attrs['href']
    article_id = link.split('=')[1].split('&')[0]
    office_id = link.split('office_id=')[1].split('&')[0]
    link = f'https://n.news.naver.com/mnews/article/{office_id}/{article_id}'
    
    data.append({"title":title, "summary":summary, "press":press, "wdate":wdate, "link":link})



In [31]:
df = pd.DataFrame(data)
df

,title,summary,press,wdate,link
0,"뉴욕증시, 예상밑돈 1분기 GDP에 급락 출발",정선영 연합인포맥스 특파원 = 뉴욕증시는 월가 예상치를 크게 밑돈 1분기 미국 국내...,연합뉴스,2024-04-25 23:28:22,https://n.news.naver.com/mnews/article/001/001...
1,"뉴욕증시, 스태그플레이션 우려에 급락…나스닥 1.9% ↓",미국 뉴욕증시의 3대 지수는 25일(현지시간) 장 초반 일제히 급락하고 있다. 미국...,아시아경제,2024-04-25 23:13:26,https://n.news.naver.com/mnews/article/277/000...
2,미국 '예상 밖' 성장 둔화·물가 급등…주식·채권 동반 급락,"미국 경제성장률이 인플레이션으로 인한 소비자, 정부 지출 여파로 2022년 2분기 ...",한국경제TV,2024-04-25 22:21:50,https://n.news.naver.com/mnews/article/215/000...
3,"美 1분기 성장률 예상 하회, 물가는 급등…2년물 국채 금리 5% 돌파",미국의 올해 1분기 경제 성장률이 예상보다 큰 폭으로 둔화됐지만 인플레이션은 가속화...,아시아경제,2024-04-25 22:20:27,https://n.news.naver.com/mnews/article/277/000...
4,제주항공 빠진 아시아나 화물매각…‘우협 2자리’ 누가 꿰찰까[마켓인],아시아나항공(020560) 화물사업부 매각을 위한 본입찰이 제주항공의 불참 속에 진...,이데일리,2024-04-25 21:12:01,https://n.news.naver.com/mnews/article/018/000...
5,"英노동당 ""집권시 철도 다시 국영화""",영국 제1야당 노동당이 올해 예정된 총선에서 집권에 성공하면 약 30년 만에 철도를...,아시아경제,2024-04-25 21:00:41,https://n.news.naver.com/mnews/article/277/000...
6,"“기대감 가득 주식창 열었는데”…호실적에도 주가는 뚝뚝, 이게 무슨 일",메타·IBM·램리서치 기대이상 실적 “AI투자 결국엔 비용부담” 전망에 美 빅테크 ...,매일경제,2024-04-25 20:58:01,https://n.news.naver.com/mnews/article/009/000...
7,"美, 마이크론에 반도체 보조금 8.4조원… ""일자리 7만개 만들 것""(종합)",미국 백악관이 25일(현지시간) 반도체법에 따라 미 최대 메모리 반도체 제조업체인 ...,아시아경제,2024-04-25 20:35:13,https://n.news.naver.com/mnews/article/277/000...
8,내수 '깜짝 성장'에 1분기 GDP 1.3%↑... 지속 여부는 '미지수',1분기(1~3월) 한국 경제가 강한 성장세를 기록했다. 내수(소비와 투자)의 예상 ...,한국일보,2024-04-25 20:01:01,https://n.news.naver.com/mnews/article/469/000...
9,"""행동주의펀드, 저평가 회사 조정… 연기금 수익자가 수혜"" [FIND 제22회 서...",행동주의 펀드는 단순히 주가를 올리는 역할만 하는 것이 아니라 저평가 회사를 조정하...,파이낸셜뉴스,2024-04-25 19:35:02,https://n.news.naver.com/mnews/article/014/000...


In [ ]:
suffix = f'{date}_{int(time.time())}'
